In [ ]:
import sqlite3

In [ ]:
DB_NAME = "biblio"

# Sqlite3

## Connection. CREATE & INSERT

In [ ]:
connection = sqlite3.connect(DB_NAME)

In [ ]:
cursor = connection.cursor()

In [ ]:
cursor.execute(
    '''
    CREATE TABLE Book(
        id INT PRIMARY KEY NOT NULL,
        title TEXT,
        author TEXT,
        publish_year INT
    )
    '''
)

cursor.execute(
    '''
    INSERT INTO Book VALUES
    (1, 'B1', 'A1', 1970),
    (2, 'B2', 'A2', 2000),
    (3, 'B3', 'A3', 2019),
    (4, 'B4', 'A4', 2005),
    (5, 'B5', 'A5', 1983)
    '''
)

In [ ]:
connection.commit()

In [ ]:
connection.close()

## SELECT

In [ ]:
connection = sqlite3.connect(DB_NAME)
cursor = connection.cursor()

In [ ]:
cursor.execute("SELECT * FROM Book WHERE publish_year > 1970")

print(cursor.fetchone())

for row in cursor.execute("SELECT * FROM Book ORDER BY publish_year"):
    print(row)

In [ ]:
connection.close()

## INSERT (again) + DELETE

In [ ]:
connection = sqlite3.connect(DB_NAME)
cursor = connection.cursor()

In [ ]:
cursor.execute(
    '''
    CREATE TABLE Reader(
        id INT PRIMARY KEY NOT NULL,
        name TEXT
    )
    '''
)

In [ ]:
persons = [
    (1, "Hugo"),
    (2, "Klein")
]

connection.executemany(
    "INSERT INTO Reader VALUES (?, ?)",
    persons
)

In [ ]:
for row in connection.execute("SELECT name FROM Reader"):
    print(row)

In [ ]:
print(
    "I just deleted",
    connection.execute("DELETE FROM Reader").rowcount,
    "rows"
)

In [ ]:
connection.close()

## SELECT (again)

In [ ]:
connection = sqlite3.connect(DB_NAME)
cursor = connection.cursor()

In [ ]:
person_id = 3
person_name = 'Johnny'

# This is the qmark style:
cursor.execute("INSERT INTO Reader VALUES (?, ?)", (person_id, person_name))

# And this is the named style:
cursor.execute(
    "SELECT * FROM Reader WHERE name=:person_name",
    {"person_name": person_name}
)

In [ ]:
print(cursor.fetchone())

In [ ]:
connection.close()

## Context Manager for Commit/Rollback

In [ ]:
connection = sqlite3.connect(DB_NAME)

In [ ]:
person_id = 5
person_name = 'Masha'

In [ ]:
with connection:
    connection.execute("INSERT INTO Reader VALUES (?, ?)", (person_id, person_name))

In [ ]:
try:
    with connection:
        connection.execute("INSERT INTO Reader VALUES (?, ?)", (person_id, person_name))
except sqlite3.IntegrityError:
    print(f"Can't add {person_name} twice")

In [ ]:
connection.close()

## Custom Object as a SELECT Result

In [ ]:
connection = sqlite3.connect(DB_NAME)

In [ ]:
connection.row_factory = sqlite3.Row

cursor = connection.cursor()

cursor.execute("SELECT * FROM Book")
row = cursor.fetchone()

print(f'Row keys: {row.keys()}')

values = [row[k] for k in row.keys()]

print(f'Row values: {values}')

In [ ]:
row

In [ ]:
connection.close()

# More Data for the DB!

In [ ]:
connection = sqlite3.connect(DB_NAME)
cursor = connection.cursor()

cursor.execute(
    '''
    CREATE TABLE Book(
        id INT PRIMARY KEY NOT NULL,
        title TEXT,
        author TEXT,
        publish_year INT
    )
    '''
)

cursor.execute(
    '''
    INSERT INTO Book VALUES
    (1, 'Harry Potter and the Philosopher''s Stone', 'J. K. Rowling', 1997),
    (2, 'Методы решения задач в общем курсе физики, Механика', 'Корявов В.П.', 2007),
    (3, 'Batman: Gotham by Gaslight', 'Brian Augustyn, Mike Mignola', 1989),
    (4, 'Мёртвые души', 'Н. В. Гоголь', 1842),
    (5, 'The Running Man', 'Richard Bachman', 1982)
    '''
)

connection.commit()

In [ ]:
cursor.execute(
    '''
    CREATE TABLE Reader(
        id INT PRIMARY KEY NOT NULL,
        name TEXT
    )
    '''
)

cursor.execute(
    '''
    INSERT INTO Reader VALUES
    (1, 'V.V. Petrovich'),
    (2, 'Alexanderr'),
    (3, 'Arkadiy Svidrigaylov'),
    (4, 'Vitalya'),
    (5, 'Noname')
    '''
)

In [ ]:
cursor.execute(
    '''
    CREATE TABLE Record(
        id INT PRIMARY KEY NOT NULL,
        reader_id INT REFERENCES Readers(id) NOT NULL,
        book_id INT REFERENCES Books(id) NOT NULL,
        taking_date TEXT,
        returning_date TEXT
    )
    '''
)

In [ ]:
cursor.execute(
    '''
    INSERT INTO Record VALUES
    (1, 3, 4, '1842-02-14', '1842-07-01'),
    (2, 4, 4, '1996-10-26', NULL),
    (3, 1, 2, '2006-06-13', '2006-06-14'),
    (4, 1, 1, '2006-06-14', '2006-06-15'),
    (5, 1, 3, '2006-06-15', '2006-07-20'),
    (6, 2, 5, '1985-10-11', '1987-12-25'),
    (7, 5, 3, '2001-03-15', NULL),
    (8, 5, 3, '2002-09-19', '2002-09-25')
    '''
)

In [ ]:
connection.commit()
connection.close()

# MongoDB

In [ ]:
from pymongo import MongoClient

In [ ]:
DB_NAME = 'biblio'

Чтобы использовать MongoDB, надо её установить и запустить (при создании `MongoClient` надо указать host и port — по умолчанию это значения `localhost` и `27017` соответственно).

In [ ]:
client = MongoClient(host='localhost')
database = client[DB_NAME]

In [ ]:
collection_name = 'Book'
collection = database[collection_name]

In [ ]:
COL_BOOK_ID = 'id'
COL_BOOK_TITLE = 'title'
COL_BOOK_AUTHOR = 'author'
COL_BOOK_PUBLISH_YEAR = 'publish_year'

In [ ]:
book = {
    COL_BOOK_ID: 2,
    COL_BOOK_TITLE: 'Harry',
    COL_BOOK_AUTHOR: 'AUTHOR',
    COL_BOOK_PUBLISH_YEAR: '1970'
}

collection.insert_one(book)

(Можно повторить ячейку выше — ошибки не будет. Внутренние "Mongo-ID" будут разными, но остальные поля у двух книжек будут идентичными.)

In [ ]:
for item in collection.find({}):
    print(item)

In [ ]:
collection.find_one({COL_BOOK_ID: 1})

In [ ]:
list(collection.find({COL_BOOK_ID: 1}))

In [ ]:
collection.update_one(
    {COL_BOOK_ID: 2},
    {"$set": {
        COL_BOOK_TITLE: 'New Title'
    }}
)

In [ ]:
collection.find_one({COL_BOOK_ID: 2})

In [ ]:
collection.delete_one({COL_BOOK_ID: 1})

In [ ]:
list(collection.find({COL_BOOK_ID: 1}))